In [8]:
import pandas as pd
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import mysql.connector
import time

# 데이터 준비 - 제목과 장르를 포함한 예시 데이터셋
data = {
    'event_name': [
        '클래식 피아노 공연',
        '재즈 밴드 콘서트',
        '미술 작품 전시회',
        '뮤지컬 "오페라의 유령"',
        '락 콘서트 라이브',
        '현대미술 특별전시',
        '연극 "햄릿"',
        '첼로 연주회'
    ],
    'genre': [
        '클래식', 
        '콘서트', 
        '전시행사', 
        '뮤지컬연극', 
        '콘서트', 
        '전시행사', 
        '뮤지컬연극', 
        '클래식'
    ]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# 데이터 확인
print(df)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# 데이터 분리 (입력과 레이블)
X = df['event_name']
y = df['genre']

# 훈련 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 텍스트 데이터를 숫자로 변환 (Bag of Words 모델 사용)
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

from sklearn.naive_bayes import MultinomialNB

# Naive Bayes 모델 초기화 및 학습
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# 모델 저장
import joblib
joblib.dump((model, vectorizer), 'genre_classifier.pkl')

from sklearn.metrics import accuracy_score, classification_report

# 테스트 데이터에 대해 예측
y_pred = model.predict(X_test_vec)

# 정확도 평가
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=1))

# 모델과 벡터라이저 로드
model, vectorizer = joblib.load('genre_classifier.pkl')

# Selenium 크롤링 코드와 장르 예측 통합
# ChromeDriverManager를 사용하여 크롬 드라이버 자동 설치 및 경로 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 열고자 하는 웹페이지 URL
driver.get("http://ticket.yes24.com/New/Notice/NoticeMain.aspx")

time.sleep(1)

# 오픈티켓목록
tickets = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#BoardList > div > table > tbody > tr")))

time.sleep(1)

for ticket in tickets[1:]:
    event_name = ticket.find_element(By.CSS_SELECTOR, 'td:nth-child(2) > a').text
    print("제목: " + event_name)
    
        # 장르 예측
    event_name_vec = vectorizer.transform([event_name])
    predicted_genre = model.predict(event_name_vec)[0]
    print(f"예측된 장르: {predicted_genre}")

    # 추가적인 크롤링 및 데이터 수집 로직...

# 웹 드라이버 종료
driver.quit()



      event_name  genre
0     클래식 피아노 공연    클래식
1      재즈 밴드 콘서트    콘서트
2      미술 작품 전시회   전시행사
3  뮤지컬 "오페라의 유령"  뮤지컬연극
4      락 콘서트 라이브    콘서트
5      현대미술 특별전시   전시행사
6        연극 "햄릿"  뮤지컬연극
7         첼로 연주회    클래식
Accuracy: 50.00%

Classification Report:
               precision    recall  f1-score   support

       뮤지컬연극       0.00      1.00      0.00         0
        전시행사       1.00      0.00      0.00         1
         콘서트       1.00      1.00      1.00         1

    accuracy                           0.50         2
   macro avg       0.67      0.67      0.33         2
weighted avg       1.00      0.50      0.50         2

제목: 단독판매 제1258회 서초금요음악회 [서초국악축제] 티켓오픈안내
예측된 장르: 뮤지컬연극
제목: 단독판매 LUCY DAYS 티켓 오픈 안내
예측된 장르: 뮤지컬연극
제목: 단독판매 너드커넥션 연말공연 〈THE NERDCONNECTION 2024〉 티켓 오픈 안내
예측된 장르: 뮤지컬연극
제목: [부천] 2025 연극 〈친정엄마와 2박3일〉- 티켓오픈 안내
예측된 장르: 뮤지컬연극
제목: 2024’ 옥희 50주년 디너 콘서트 티켓 오픈 안내
예측된 장르: 콘서트
제목: 로열 클래식 발레단 내한공연〈호두까기 인형〉- 부산 티켓오픈 안내
예측된 장르: 클래식
제목: 단독판매 2024 먼데이키즈 소극장 콘서트 ‘ㅇㅈㅅ’ 티켓오픈 안내
예측

In [11]:
import pandas as pd
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import mysql.connector
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib

# 데이터 준비 - 기존 데이터셋
data = {
    'event_name': [
        '클래식 피아노 공연',
        '재즈 밴드 콘서트',
        '미술 작품 전시회',
        '뮤지컬 "오페라의 유령"',
        '락 콘서트 라이브',
        '현대미술 특별전시',
        '연극 "햄릿"',
        '첼로 연주회',
        '조수미＆요한슈트라우스 오케스트라 울산공연 티켓오픈안내',
        '단독판매 너드커넥션 연말공연 〈THE NERDCONNECTION 2024〉 티켓 오픈 안내',
        '단독판매 A New Hope Club Christmas in Seoul 티켓 오픈 안내',
        '단독판매 [부산] YEONGDO INDIE CONCERT [로맨틱 영도] 티켓 오픈 안내',
        '단독판매 정홍일 Only One Concert - 티켓오픈 안내',
        '사라 장 바이올린 리사이틀 티켓오픈 안내',
        '뮤지컬 <카포네 밀크> 1차 티켓오픈 안내'
    ],
    'genre': [
        '클래식', 
        '콘서트', 
        '전시행사', 
        '뮤지컬연극', 
        '콘서트', 
        '전시행사', 
        '뮤지컬연극', 
        '클래식',
        '클래식',
        '콘서트',
        '콘서트',
        '콘서트',
        '콘서트',
        '클래식',
        '뮤지컬'
    ]
}

# 데이터프레임 생성
df = pd.DataFrame(data)

# Selenium 크롤링 코드와 장르 예측 통합
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get("https://ticket.melon.com/csoon/index.htm")

time.sleep(1)

new_event_names = []
new_genres = []

categories = [
    ("콘서트", "#sForm > div > div.wrap_form_input.box_ticket_search > div > div > div > ul > li:nth-child(2) > a"),
    ("뮤지컬연극", "#sForm > div > div.wrap_form_input.box_ticket_search > div > div > div > ul > li:nth-child(3) > a"),
    ("클래식", "#sForm > div > div.wrap_form_input.box_ticket_search > div > div > div > ul > li:nth-child(5) > a"),
    ("전시행사", "#sForm > div > div.wrap_form_input.box_ticket_search > div > div > div > ul > li:nth-child(6) > a")
]

for category_name, category_selector in categories:
    driver.execute_script(f"document.querySelector('{category_selector}').click();")
    time.sleep(2)

    for page in range(1, 11):  # 최대 10페이지 반복
        try:
            # 페이지 이동
            if page > 1:
                driver.execute_script(f"goPage('{(page - 1) * 10 + 1}');")
                time.sleep(2)

            tickets = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.list_ticket_cont > li')))
            
            for ticket in tickets:
                try:
                    event_name = ticket.find_element(By.CSS_SELECTOR, 'div.link_consert > a').text
                    print("제목: " + event_name)
                    genre = category_name
                    new_event_names.append(event_name)
                    new_genres.append(genre)
                except Exception as e:
                    print(f"Error during crawling: {e}")
                    continue
        except TimeoutException:
            print(f"No more data found on page {page} for category {category_name}.")
            break

# 웹 드라이버 종료
driver.quit()

new_data = pd.DataFrame({
    'event_name': new_event_names,
    'genre': new_genres
})
df = pd.concat([df, new_data], ignore_index=True)

X = df['event_name']
y = df['genre']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_vec, y_train)

joblib.dump((model, vectorizer), 'genre_classifier.pkl')

y_pred = model.predict(X_test_vec)

from sklearn.metrics import accuracy_score, classification_report
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=1))

print("모델 업데이트 완료!")

제목: NCT 127 4TH TOUR ‘NEO CITY : SEOUL - THE MOMENTUM’ 티켓 오픈 안내
제목: 2024 김사월 연말 단독 공연 〈밖은 너무 추워 나는〉 티켓 오픈 안내
제목: 2024 서울시티뮤직페스티벌 티켓 오픈 안내
제목: 성휘 단독 콘서트 “사랑이 많은 밤” 티켓 오픈 안내
제목: 먼데이프로젝트 시즌7 ORIGINAL [Red C (레드씨) 단독 콘서트] 티켓 오픈 안내
제목: My aunt mary : SING 티켓 오픈 안내
제목: 화노 단독 콘서트 [무아(舞雅) : 아름다운 춤] 티켓 오픈 안내
제목: 연정(YEONJEONG) 첫번째 단독콘서트 [눈에 보이지 않는 노래는] 티켓 오픈 안내
제목: 우리집 음악회 티켓 오픈 안내
제목: Song Sohee 1st Concert：風流（풍류）티켓 오픈 안내
제목: 아야츠노 유니 1st solo concert “Christmas On The Merry Go Round" 티켓 오픈 안내
제목: 에이퍼즈 앨범발매 쇼케이스 티켓 오픈 안내
제목: 2024 KAVE 단독 콘서트 투어 〈Say My Name〉 서울, 부산 티켓 오픈 안내
제목: 먼데이프로젝트 시즌7 [김바다밴드 단독 콘서트 ‘바다가 시작되는 곳’] 티켓 오픈 안내
제목: 2024 B-day PARTY - CHANYEOL [OFF THE RECORD] 티켓 오픈 안내
제목: NO HIPHOP NO LIFE (노힢노랖) Vol.003 - 언에듀케이티드 키드(UNEDUCATED KID) 티켓 오픈 안내
제목: 2024 케이시 콘서트 〈아홉번째 겨울: 사랑의 편린〉 티켓 오픈 안내
제목: 그리고 영신 티켓 오픈 안내
제목: 2024 다양성 연말 단독 공연 〈SNOWBALL〉 티켓 오픈 안내
제목: 마리슈 10주년 콘서트 ‘10’ 티켓 오픈 안내
제목: 제70회 라이브 클럽 데이－블라인드 티켓 오픈 안내
제목: 정효빈 소극장 단독 콘서트 [The Blossom] 티켓 오픈 안내
제목: 폴카이트 X 최인경 겨울 콘서트 ‘12:01’ 티

In [12]:
import joblib

# 모델과 벡터라이저 로드
model, vectorizer = joblib.load('genre_classifier.pkl')

# 모델 정보 출력
print("모델 정보:")
print(model)

# 벡터라이저 정보 출력
print("\n벡터라이저 정보:")
print(vectorizer)

# 벡터라이저의 단어 목록 확인
print("\n벡터라이저에 의해 학습된 단어 목록:")
print(vectorizer.get_feature_names_out())

모델 정보:
MultinomialNB()

벡터라이저 정보:
CountVectorizer()

벡터라이저에 의해 학습된 단어 목록:
['01' '10' '100주년' '101' '10월' '10주년' '11시' '11월' '11월이' '12' '127' '1st'
 '1주년' '1차' '1회' '2000' '2018' '2020' '2020세종시즌' '2021' '2022' '2023'
 '2024' '2024_셀러브리티' '2024년' '2025' '24' '25' '2nd' '2집' '2차' '30주년' '3rd'
 '3기' '3차' '3층' '4월은' '4차' '50주년' '57분' '5th' '5차' '6개의' '6주년' '6차' '70주년'
 '88댕댕픽' '8th' '9와' '_싱크넥스트24' 'again' 'album' 'american' 'ampers' 'andor'
 'anotherverse' 'apink' 'arch' 'archive' 'art' 'artist' 'asia' 'at' 'aunt'
 'auto' 'autumn' 'ball' 'band' 'banksy' 'berklee' 'beyond' 'blasé'
 'blossom' 'blue' 'boss' 'break' 'busan' 'by' 'chanson' 'chanyeol' 'check'
 'child' 'christmas' 'chronicles' 'city' 'cj' 'club' 'color' 'colours'
 'compas' 'concert' 'conference' 'connection' 'coomo' 'cracker' 'daldam'
 'day' 'ddbb' 'de' 'dear' 'delivery' 'desire' 'dg120주년' 'di' 'documented'
 'dream' 'dreamcatcher' 'duo' 'eclipse' 'emotional' 'ep' 'ep발매' 'escape'
 'españa' 'eternal' 'exhibit' 'exhibition' 'fact'